In [1]:
import json
import pandas as pd
import numpy as np
import re

In [2]:
file_dir = 'C://Users/fenwa/Desktop/Git/Movies-ETL/'

In [3]:
with open(f'{file_dir}wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)

In [4]:
len(wiki_movies_raw)

7311

In [5]:
#first 5 records
#wiki_movies_raw[:5]

In [6]:
# Last 5 records
#wiki_movies_raw[-5:]

In [7]:
# Some records in the middle
#wiki_movies_raw[3600:3605]

In [8]:
kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir}ratings.csv')

In [9]:
wiki_movies_df = pd.DataFrame(wiki_movies_raw)

In [10]:
#wiki_movies_df.head()

In [11]:
#wiki_movies_df.columns.tolist()

In [12]:
#[expression for element in source_list]
#[expression for element in source_list if filter_expression]
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]

In [13]:
wiki_movies_only_df = pd.DataFrame(wiki_movies)

In [14]:
#wiki_movies_only_df.head()

In [15]:
#wiki_movies_only_df.columns.tolist()

In [16]:
#we can remove no. episodes because it's only for tv. 

In [17]:
#moving this process to a function to make it easier
#Lambda = one line functions; lambda arguments: expression
#def + : makes a function
#def clean_movie(movie):
    # make a copy of movie so it is non-destructive
    # the 'movie' below is a local variable and doenst mess with the global movie variable!
    #movie = dict(movie)
    #return movie

In [18]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]['url']

7060    https://en.wikipedia.org/wiki/The_Insult_(film)
7293     https://en.wikipedia.org/wiki/Capernaum_(film)
Name: url, dtype: object

In [19]:
#sorted(wiki_movies_df.columns.tolist())

In [20]:
## born/died/family/gender/parents/parent(s) is probably for people and not relevant to movies - maybe remove later?
## engines is probably for video games
#Also known as
#arabic
#bopomofo - Chinese
#Cantonese
#Chinese
#French
#Gwoyeu Romatzyh - more Chinese
#Hangul - Korean
#Hanyu Pinyin - Chinese
#Hebrew
#Hepburn - Japanese
#Hokkien POJ - Taiwanese Chinese
#IPA Chinese
#Japanese
#Jyutping - Chinese
#Mandarin
#McCune-Reischaur
#Polish
#Revised Romanization - Korean
#Romanized - unlclear which language is being Romanized
#Russian
#Simplified - Probably Chinese
#Simplified Chinese
#Traditional - Probably Chinese
#Traditional Chinese
#Wade-Giles - Chinese
#Yale Romanization - Chinese
#Yiddish


In [21]:
# def clean_movie(movie):
#     movie = dict(movie)
#     alt_titles = {}
#     for key in ['Also known as', 'Arabic', 'Bopomofo','Cantonese','Chinese','French',
#                 'Gwoyeu Romatzyh','Hangul','Hebrew','Hokkien POJ', 'IPA Chinese',
#                 'Japanese','Jyutping','Mandarin','McCune–Reischauer','Polish',
#                 'Revised Romanization','Romanized','Russian','Simplified',
#                 'Simplified Chinese', 'Traditional','Traditional Chinese',
#                 'Wade-Giles','Yale Romanization','Yiddish', 'Hepburn']:
#         if key in movie:
#             alt_titles[key] = movie[key]
#             movie.pop(key)
#     if len(alt_titles) > 0:
#         movie['alt_titles'] = alt_titles
#     return movie

In [22]:
clean_movies = [clean_movie(movie) for movie in wiki_movies]

NameError: name 'clean_movie' is not defined

In [ ]:
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

['Adaptation by',
 'Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Length',
 'Literally',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Original title',
 'Picture format',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Running time',
 'Screen story by',
 'Screenplay by',
 'Starring',
 'Story by',
 'Suggested by',
 'Theme music composer',
 'Venue',
 'Voices of',
 'Written by',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']

In [23]:
wiki_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990.0,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
#wiki_movies_df['alt_titles'].notnull()

KeyError: 'alt_titles'

In [64]:
# def change_column_name(old_name, new_name):
#     if old_name in movie:
#         movie[new_name] = movie.pop(old_name)

In [65]:
# wiki_movies_df.columns.tolist()

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Preceded by',
 'Adaptation by',
 'Suggested by',
 'alt_titles',
 'Released',
 'Recorded',
 'Venue',
 'Length',
 'Label',
 'Director',
 'Producer',
 'Animation by',
 'Color process',
 'Screen story by',
 'Original title',
 'Literally']

In [66]:
#directed by, director
#Adaptation by, Writer(s)
#Country of origin, Country
#produced by, producer(s)
#producer, producer(s)
#screenplay by, writer(s)
#story by, writer(s)
#narrated by, narrator(s)
#music by, composer(s)
#productioncompany, production company(s)
#productioncompanies, production company(s)
#edited by, editor(s)
#distributed by, distributor
#written by, writer(s)
#Theme music composer, composer(s)
#original release, release date
#released, Release Date
#length, running time
#screen story by writer(s)


In [25]:
def clean_movie(movie):
    movie = dict(movie)
    alt_titles = {}
    for key in ['Also known as', 'Arabic', 'Bopomofo','Cantonese','Chinese','French',
                'Gwoyeu Romatzyh','Hangul','Hebrew','Hokkien POJ', 'IPA Chinese',
                'Japanese','Jyutping','Mandarin','McCune–Reischauer','Polish',
                'Revised Romanization','Romanized','Russian','Simplified',
                'Simplified Chinese', 'Traditional','Traditional Chinese',
                'Wade-Giles','Yale Romanization','Yiddish', 'Hepburn']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles   
    #merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)

    change_column_name('Directed by','Director')
    change_column_name('Adaptation by','Writer(s)')
    change_column_name('Screenplay by','Writer(s)')
    change_column_name('Story by','Writer(s)')
    change_column_name('Written by','Writer(s)')
    change_column_name('Screen story by','Writer(s)')
    change_column_name('Country of origin','Country')
    change_column_name('Produced by','Producer(s)')
    change_column_name('Producer','Producer(s)')
    change_column_name('Narrated by','Narrator(s)')
    change_column_name('Music by','Composer(s)')
    change_column_name('Theme music composer','Composer(s)')
    change_column_name('Productioncompany ','Production Company(s)')
    change_column_name('Productioncompanies ','Production Company(s)')
    change_column_name('Production company(s)','Production Company(s)')
    change_column_name('Edited by','Editor(s)')
    change_column_name('Distributed by','Distributor(s)')
    change_column_name('Distributor','Distributor(s)')
    change_column_name('Original Release', 'Release Date')
    change_column_name('Released','Release Date')
    change_column_name('Release date','Release Date')
    change_column_name('Length','Running Time')
    change_column_name('Running time','Running Time')

    return movie



In [26]:
clean_movies = [clean_movie(movie) for movie in wiki_movies]
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

['Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Created by',
 'Director',
 'Distributor(s)',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Literally',
 'Narrator(s)',
 'Original language(s)',
 'Original network',
 'Original release',
 'Original title',
 'Picture format',
 'Preceded by',
 'Producer(s)',
 'Production Company(s)',
 'Production location(s)',
 'Recorded',
 'Release Date',
 'Running Time',
 'Starring',
 'Suggested by',
 'Venue',
 'Voices of',
 'Writer(s)',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']

In [27]:
wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wiki_movies_df))
wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
print(len(wiki_movies_df))
wiki_movies_df.head()

7076
7033


,url,year,imdb_link,title,Based on,Starring,Cinematography,Country,Language,Budget,...,Suggested by,alt_titles,Recorded,Venue,Label,Animation by,Color process,Original title,Literally,imdb_id
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,United States,English,$20 million,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0098987
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,United States,English,$6 million,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0098994
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,United States,"[English, Lao]",$35 million,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099005
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,United States,English,$12 million,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099012
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,US,English,$25 million,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099018


In [28]:
# [[column,wiki_movies_df[column].isnull().sum() for column in wiki_movies_df.columns < len(wiki_movies_df) * .9]

SyntaxError: invalid syntax (648431518.py, line 1)

In [84]:
[column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]


['url',
 'year',
 'imdb_link',
 'title',
 'Based on',
 'Starring',
 'Cinematography',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Director',
 'Writer(s)',
 'Producer(s)',
 'Composer(s)',
 'Production Company(s)',
 'Editor(s)',
 'Distributor(s)',
 'Release Date',
 'Running Time',
 'imdb_id']

In [29]:
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

In [30]:
wiki_movies_df.dtypes

url                      object
year                      int64
imdb_link                object
title                    object
Based on                 object
Starring                 object
Cinematography           object
Country                  object
Language                 object
Budget                   object
Box office               object
Director                 object
Writer(s)                object
Producer(s)              object
Composer(s)              object
Production Company(s)    object
Editor(s)                object
Distributor(s)           object
Release Date             object
Running Time             object
imdb_id                  object
dtype: object

In [39]:
#box office should be numeric
#budget should be numeric
#release date should be date
#running time should numeric

box_office = wiki_movies_df['Box office'].dropna()

In [40]:

def is_not_a_string(x):
    return type(x) != str

box_office[box_office.map(is_not_a_string)]

34                           [US$, 4,212,828]
54      [$6,698,361 (, United States, ), [2]]
74                    [$6,488,144, (US), [1]]
126                [US$1,531,489, (domestic)]
130                          [US$, 4,803,039]
                        ...                  
6980               [$99.6, million, [4], [5]]
6994                   [$365.6, million, [1]]
6995                         [$53.8, million]
7015                     [$435, million, [7]]
7048                   [$529.3, million, [4]]
Name: Box office, Length: 135, dtype: object

In [41]:
box_office[box_office.map(lambda x: type(x) != str)]


34                           [US$, 4,212,828]
54      [$6,698,361 (, United States, ), [2]]
74                    [$6,488,144, (US), [1]]
126                [US$1,531,489, (domestic)]
130                          [US$, 4,803,039]
                        ...                  
6980               [$99.6, million, [4], [5]]
6994                   [$365.6, million, [1]]
6995                         [$53.8, million]
7015                     [$435, million, [7]]
7048                   [$529.3, million, [4]]
Name: Box office, Length: 135, dtype: object

In [42]:
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)


In [ ]:
# \d will match any digit from 0 to 9.
# \D will match any non-digit character.
# \w matches a word character (a letter, digit, or underscore).
# \W matches any non-word character (anything other than a letter, digit, or underscore, such as spaces and punctuation).
# \s will match any whitespace character (including spaces, tabs, and newlines).
# \S will match any non-whitespace characters.

In [ ]:
# literal characters	Directly matches characters	"cat"
# “cat”
# “dog” (no match)
# \d	Matches a digit from 0 to 9	"\d"
# “1”
# “A” (no match)
# “_” (no match)
# “!” (no match)
# “ ” (no match)
# \D	Matches a non-digit	"\D"
# “1” (no match)
# “A”
# “_”
# “!”
# “ ”
# \w	Matches a word character (letter, digit, or underscore)	"\w"
# “1”
# “A”
# “_”
# “!” (no match)
# “ ” (no match)
# \W	Matches any non-word character	"\W"
# “1” (no match)
# “A” (no match)
# “_” (no match)
# “!”
# “ ”
# \s	Matches any whitespace character, such as spaces and tabs	"\s"
# “1” (no match)
# “A” (no match)
# “_” (no match)
# “!” (no match)
# “ ”
# \S	Matches any non-whitespace character	\S
# “1”
# “A”
# “_”
# “!”
# “ ” (no match)
# [ … ]	Character Set
# Matches any characters inside the brackets. Can specify ranges of characters as well.

# "[A-C]"
# “A”
# “B”
# “C”
# “D” (no match)
# “E” (no match)
# [^ … ]	Negative Character Set
# Matches anything not inside the brackets

# "[^C-E]"
# “A”
# “B”
# “C” (no match)
# “D” (no match)
# “E” (no match)
# .	Wildcard
# Matches any character (except a newline)

# "."
# “1”
# “A”
# “_”
# “!”
# “ ”
# *	Matches 0 or more times	"ca*t"
# “ct”
# “cat”
# “caat”
# “caaat”
# “caaaat”
# +	Matches 1 or more times	"ca+t"
# “ct”
# “cat”
# “caat”
# “caaat”
# “caaaat”
# ?	Matches 0 or 1 time	"ca?t"
# “ct”
# “cat”
# “caat”
# “caaat”
# “caaaat”
# {#}	Matches a specific number of times	"ca{2}t"
# “ct”
# “cat”
# “caat”
# “caaat”
# “caaaat”
# {#,}	Matches at least a specific number of times	"ca{2,}t"
# “ct”
# “cat”
# “caat”
# “caaat”
# “caaaat”
# {#,#}	Matches within a specific range of times	"ca{2,3}t"
# “ct”
# “cat”
# “caat”
# “caaat”
# “caaaat”
# |	Alternation
# Matches either the expression before or the expression after

# "cat|dog"
# “cat”
# “dog”
# “bird”
# ^	Start of the string	"^cat"
# “cat”
# “catsup”
# “concatenate” (no match)
# “kitty-cat” (no match)
# $	End of the string	"cat$"
# “cat”
# “catsup” (no match)
# “concatenate” (no match)
# “kitty-cat”
# </code>	Escape Character
# Escapes the next character to be treated as a literal character

# "\$"
# “$”
# ( … )	Capture Group
# Identifies matches that should be extracted

# "c(at)"
# “cat” (“at” is captured)
# “bat” (no match)
# (?: … )	Non-Capturing Group
# Identifies matches that should not be extracted

# "c(?:at)"
# “cat” ("c" is captured)
# “bat” (no match)
# (?! … )	Negative Lookahead Group
# Identifies expressions that negate earlier matches

# "cat(?! burglar)"
# “cat”
# “cats”
# # “cat burglar” (no match)

In [70]:
form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'

In [71]:
box_office.str.contains(form_one, flags=re.IGNORECASE, na=False).sum()

3909

In [72]:
form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illi?on)'

In [73]:
box_office.str.contains(form_two, flags=re.IGNORECASE, na=False).sum()

1559

In [74]:
matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE, na=False)
matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE, na=False)

In [75]:
box_office = box_office.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
box_office[~matches_form_one & ~matches_form_two]

600                      $5000 (US)
1070                     35,254,617
1480                     £3 million
1865                   ¥1.1 billion
2032                            N/A
2091                           $309
2665    926,423 admissions (France)
3631                            TBA
3879       CN¥3.650 million (China)
4116                     £7,385,434
4306                            $30
4561        $45.2k (only in Turkey)
5447                          £2.56
5784                       413 733$
6013                        Unknown
6369                          $111k
6370                           $588
6593                 less than $372
6843                        8 crore
Name: Box office, dtype: object

In [76]:
box_office.str.extract(f'({form_one}|{form_two})')


,0
0,$21.4 million
1,$2.7 million
2,"$57,718,089"
3,"$7,331,647"
4,"$6,939,946"
...,...
7070,$19.4 million
7071,$41.9 million
7072,$76.1 million
7073,$38.4 million


In [78]:
def parse_dollars(s):
    # if s is not a string, return NaN
    if type(s) != str:
        return np.nan
    # if input is of the form $###.# million
    if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):
        # remove dollar sign and " million"
        s = re.sub('\$|\s|[a-zA-Z]','', s)
        # convert to float and multiply by a million
        value = float(s) * 10**6
        # return value
        return value
    # if input is of the form $###.# billion
    elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " billion"
        s = re.sub('\$|\s|[a-zA-Z]','', s)
        # convert to float and multiply by a billion
        value = float(s) * 10**9
        # return value
        return value
    # if input is of the form $###,###,###
    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):
        # remove dollar sign and commas
        s = re.sub('\$|,','', s)
        # convert to float
        value = float(s)
        # return value
        return value
    # otherwise, return NaN
    else:
        return np.nan

In [79]:
wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

In [80]:
wiki_movies_df['box_office'].dtypes

dtype('float64')

In [81]:
wiki_movies_df.drop('Box office', axis=1, inplace=True)

In [82]:
budget = wiki_movies_df['Budget'].dropna()

In [83]:
budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)

In [84]:
budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

In [87]:
matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE, na=False)
matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE, na=False)
budget[~matches_form_one & ~matches_form_two]

136                         Unknown
204     60 million Norwegian Kroner
478                         Unknown
973             $34 [3] [4] million
1126               $120 [4] million
1226                        Unknown
1278                            HBO
1374                     £6,000,000
1397                     13 million
1480                   £2.8 million
1734                   CAD2,000,000
1913     PHP 85 million (estimated)
1948                    102,888,900
1953                   3,500,000 DM
1973                     ₤2,300,874
2281                     $14 milion
2451                     ₤6,350,000
3144                   € 40 million
3360               $150 [6] million
3418                        $218.32
3802                   £4.2 million
3906                            N/A
3959                    760,000 USD
4470                       19 crore
4641                    £17 million
5034              $$200 [4] million
5055           $155 [2] [3] million
5419                $40 [4] 

In [90]:
budget = budget.str.replace(r'\[\d+\]\s*', '')
budget[~matches_form_one & ~matches_form_two]

c:\Users\fenwa\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


136                         Unknown
204     60 million Norwegian Kroner
478                         Unknown
973                     $34 million
1126                   $120 million
1226                        Unknown
1278                            HBO
1374                     £6,000,000
1397                     13 million
1480                   £2.8 million
1734                   CAD2,000,000
1913     PHP 85 million (estimated)
1948                    102,888,900
1953                   3,500,000 DM
1973                     ₤2,300,874
2281                     $14 milion
2451                     ₤6,350,000
3144                   € 40 million
3360                   $150 million
3418                        $218.32
3802                   £4.2 million
3906                            N/A
3959                    760,000 USD
4470                       19 crore
4641                    £17 million
5034                  $$200 million
5055                   $155 million
5419                    $40 

In [91]:
wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)


In [92]:
wiki_movies_df.drop('Budget', axis=1, inplace=True)

In [93]:
wiki_movies_df

,url,year,imdb_link,title,Based on,Starring,Cinematography,Country,Language,Director,...,Producer(s),Composer(s),Production Company(s),Editor(s),Distributor(s),Release Date,Running Time,imdb_id,box_office,budget
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,United States,English,Renny Harlin,...,"[Steve Perry, Joel Silver]","[Cliff Eidelman, Yello]",Silver Pictures,Michael Tronick,20th Century Fox,"[July 11, 1990, (, 1990-07-11, )]",102 minutes,tt0098987,21400000.0,20000000.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,United States,English,James Foley,...,"[Ric Kidney, Robert Redlin]",Maurice Jarre,Avenue Pictures,Howard E. Smith,Avenue Pictures,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",114 minutes,tt0098994,2700000.0,6000000.0
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,United States,"[English, Lao]",Roger Spottiswoode,...,Daniel Melnick,Charles Gross,"[Carolco Pictures, IndieProd Company]","[John Bloom, Lois Freeman-Fox]",TriStar Pictures,"[August 10, 1990, (, 1990-08-10, )]",113 minutes,tt0099005,57718089.0,35000000.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,United States,English,Woody Allen,...,Robert Greenhut,NaN,NaN,Susan E. Morse,Orion Pictures,"[December 25, 1990, (, 1990-12-25, )]",106 minutes,tt0099012,7331647.0,12000000.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,US,English,John Cornell,...,John Cornell,Maurice Jarre,NaN,David Stiven,Paramount Pictures,"December 19, 1990",95 minutes,tt0099018,6939946.0,25000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7071,https://en.wikipedia.org/wiki/Holmes_%26_Watson,2018,https://www.imdb.com/title/tt1255919/,Holmes & Watson,"[Sherlock Holmes, and, Dr. Watson, by, Sir Art...","[Will Ferrell, John C. Reilly, Rebecca Hall, R...",Oliver Wood,United States,English,Etan Cohen,...,"[Will Ferrell, Adam McKay, Jimmy Miller, Clayt...",Mark Mothersbaugh,"[Columbia Pictures, Gary Sanchez Productions, ...",Dean Zimmerman,Sony Pictures Releasing,"[December 25, 2018, (, 2018-12-25, ), (United ...",90 minutes,tt1255919,41900000.0,42000000.0
7072,https://en.wikipedia.org/wiki/Vice_(2018_film),2018,https://www.imdb.com/title/tt6266538/,Vice,NaN,"[Christian Bale, Amy Adams, Steve Carell, Sam ...",Greig Fraser,United States,English,Adam McKay,...,"[Brad Pitt, Dede Gardner, Jeremy Kleiner, Kevi...",Nicholas Britell,"[Plan B Entertainment, Gary Sanchez Production...",Hank Corwin,Mirror Releasing,"[December 11, 2018, (, 2018-12-11, ), (, Samue...",132 minutes,tt6266538,76100000.0,60000000.0
7073,https://en.wikipedia.org/wiki/On_the_Basis_of_Sex,2018,https://www.imdb.com/title/tt4669788/,On the Basis of Sex,NaN,"[Felicity Jones, Armie Hammer, Justin Theroux,...",Michael Grady,United States,English,Mimi Leder,...,Robert W. Cort,Mychael Danna,"[Focus Features, [1], Participant Media, [1], ...",Michelle Tesoro,Focus Features,"[November 8, 2018, (, 2018-11-08, ), (, AFI Fe...",120 minutes,tt4669788,38400000.0,20000000.0
7074,https://en.wikipedia.org/wiki/Destroyer_(2018_...,2018,https://www.imdb.com/title/tt7137380/,Destroyer,NaN,"[Nicole Kidman, Sebastian Stan, Toby Kebbell, ...",Julie Kirkwood,United States,English,Karyn Kusama,...,"[Fred Berger, Phil Hay, Matt Manfr

In [95]:
release_date = wiki_movies_df['Release Date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)


In [96]:
date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]?\d,\s\d{4}'
date_form_two = r'\d{4}.[01]\d.[0123]\d'
date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
date_form_four = r'\d{4}'

In [97]:
release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)


,0
0,"July 11, 1990"
1,"May 17, 1990"
2,"August 10, 1990"
3,"December 25, 1990"
4,"December 19, 1990"
...,...
7071,"December 25, 2018"
7072,"December 11, 2018"
7073,"November 8, 2018"
7074,"August 31, 2018"


In [98]:

wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)

In [101]:
wiki_movies_df.drop('Release Date', axis=1, inplace=True)

In [103]:

running_time = wiki_movies_df['Running Time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

6877

In [109]:
running_time[running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE, na=False) != True]

668                     UK:84 min (DVD version) US:86 min
727                         78-102 min (depending on cut)
840                       Varies (79 [3] –84 [1] minutes)
1347                                              25 : 03
1443    United States: 77 minutes Argentina: 94 minute...
1499                                            1hr 35min
1551                                               varies
1774                    Netherlands:96 min, Canada:95 min
1777                                       approx. 14 min
2273                                           1 h 43 min
2993                                               1h 48m
3925                                              4 hours
4425    US domestic version: 86 minutes Original versi...
4967    Theatrical cut: 97 minutes Unrated cut: 107 mi...
5424                    115 [1] /123 [2] /128 [3] minutes
5447                                    1 hour 32 minutes
7075                Variable; 90 minutes for default path
Name: Running 

In [110]:
running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')


In [111]:
running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)


In [112]:
wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)


In [114]:
wiki_movies_df.drop('Running Time', axis=1, inplace=True)


In [115]:
wiki_movies_df

,url,year,imdb_link,title,Based on,Starring,Cinematography,Country,Language,Director,...,Producer(s),Composer(s),Production Company(s),Editor(s),Distributor(s),imdb_id,box_office,budget,release_date,running_time
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,United States,English,Renny Harlin,...,"[Steve Perry, Joel Silver]","[Cliff Eidelman, Yello]",Silver Pictures,Michael Tronick,20th Century Fox,tt0098987,21400000.0,20000000.0,1990-07-11,102.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,United States,English,James Foley,...,"[Ric Kidney, Robert Redlin]",Maurice Jarre,Avenue Pictures,Howard E. Smith,Avenue Pictures,tt0098994,2700000.0,6000000.0,1990-05-17,114.0
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,United States,"[English, Lao]",Roger Spottiswoode,...,Daniel Melnick,Charles Gross,"[Carolco Pictures, IndieProd Company]","[John Bloom, Lois Freeman-Fox]",TriStar Pictures,tt0099005,57718089.0,35000000.0,1990-08-10,113.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,United States,English,Woody Allen,...,Robert Greenhut,NaN,NaN,Susan E. Morse,Orion Pictures,tt0099012,7331647.0,12000000.0,1990-12-25,106.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,US,English,John Cornell,...,John Cornell,Maurice Jarre,NaN,David Stiven,Paramount Pictures,tt0099018,6939946.0,25000000.0,1990-12-19,95.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7071,https://en.wikipedia.org/wiki/Holmes_%26_Watson,2018,https://www.imdb.com/title/tt1255919/,Holmes & Watson,"[Sherlock Holmes, and, Dr. Watson, by, Sir Art...","[Will Ferrell, John C. Reilly, Rebecca Hall, R...",Oliver Wood,United States,English,Etan Cohen,...,"[Will Ferrell, Adam McKay, Jimmy Miller, Clayt...",Mark Mothersbaugh,"[Columbia Pictures, Gary Sanchez Productions, ...",Dean Zimmerman,Sony Pictures Releasing,tt1255919,41900000.0,42000000.0,2018-12-25,90.0
7072,https://en.wikipedia.org/wiki/Vice_(2018_film),2018,https://www.imdb.com/title/tt6266538/,Vice,NaN,"[Christian Bale, Amy Adams, Steve Carell, Sam ...",Greig Fraser,United States,English,Adam McKay,...,"[Brad Pitt, Dede Gardner, Jeremy Kleiner, Kevi...",Nicholas Britell,"[Plan B Entertainment, Gary Sanchez Production...",Hank Corwin,Mirror Releasing,tt6266538,76100000.0,60000000.0,2018-12-11,132.0
7073,https://en.wikipedia.org/wiki/On_the_Basis_of_Sex,2018,https://www.imdb.com/title/tt4669788/,On the Basis of Sex,NaN,"[Felicity Jones, Armie Hammer, Justin Theroux,...",Michael Grady,United States,English,Mimi Leder,...,Robert W. Cort,Mychael Danna,"[Focus Features, [1], Participant Media, [1], ...",Michelle Tesoro,Focus Features,tt4669788,38400000.0,20000000.0,2018-11-08,120.0
7074,https://en.wikipedia.org/wiki/Destroyer_(2018_...,2018,https://www.imdb.com/title/tt7137380/,Destroyer,NaN,"[Nicole Kidman, Sebastian Stan, Toby Kebbell, ...",Julie Kirkwood,United States,English,Karyn Kusama,...,"[Fred Berger, Phil Hay, Matt Manfredi]",Theodore Shapiro,"[30West, Automatik Entertainment, Annapurna Pi...",Plummy Tucker,Mirror Releasing,tt7137380,5500000.0,9000000.0,2018-08-31,123.0
